In [1]:
from inventory import Inventory
from exception import InventoryRemoveZeroError
from entity import Entity
from sql import select_inventory

In [2]:
item = Entity("Yellow Chair")
print(item)
location = Entity("Lounge")
print(location)


Yellow Chair (total inventory: 3)
Lounge (total inventory: 1)


In [3]:
inventory_item = select_inventory(entity_id_child=item.id)
for row in inventory_item:
    print(dict(row))
print()
inventory_location = select_inventory(entity_id_parent=location.id)
for row in inventory_location:
    print(dict(row))

{'inventory_id': '4e18e23a-ca8b-4ed3-9ba6-8e2fb46d6cd8', 'inventory_id_parent': '6dc856f4-42fe-4fe0-868b-9d002eff2e27', 'entity_id_parent': 'e5998068-17a4-4c7d-b64f-4b6e28e8b93d', 'entity_id_child': '4d10faad-8a10-436b-b732-011a7b4ad043', 'quantity': 3, 'position': 'IN', 'hierarchy_level': 2, 'created_time': '2025-02-20 20:00:05'}

{'inventory_id': '2dfea04c-e8ea-4735-8b9a-4a9bf92dca08', 'inventory_id_parent': '8ce8d635-c5c5-49d7-bdcf-2b5cfb2298fe', 'entity_id_parent': '03dd5c43-d66e-4c3f-a6eb-eb67b47c0373', 'entity_id_child': 'bb85b691-61f2-4a73-8313-3e5367d4b7b6', 'quantity': 5, 'position': 'IN', 'hierarchy_level': 2, 'created_time': '2025-02-20 20:00:05'}


In [4]:
inventory = Inventory(location, item, 4)
print(inventory)

4 yellow chairs in lounge


In [5]:
# remove all yellow chairs from lounge
new_quantity = inventory.new_quantity
inventory = Inventory(location, item, (new_quantity * -1))
print(inventory.new_quantity)

0


In [6]:
# try removing a further yellow chair from the lounge
try:
    inventory = Inventory(location, item, -1)
except InventoryRemoveZeroError as e:
    print(e)

There are no yellow chairs in lounge to remove
